In [1]:
import matplotlib.pyplot as plt
import os
import torch 
from torch.utils.data import DataLoader
import json
import sys
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

sys.path.append("src")
os.chdir('/home/george-vengrovski/Documents/projects/tweety_bert_paper')

from data_class import SongDataSet_Image, CollateFunction
from model import TweetyBERT
from analysis import plot_umap_projection
from utils import detailed_count_parameters, load_weights, load_model


weights_path = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/temp_storage/10mask_500_cluster_relative/saved_weights/model_step_49995.pth"
config_path = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/temp_storage/10mask_500_cluster_relative/config.json"

tweety_bert_model = load_model(config_path, weights_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/home/george-vengrovski/anaconda3/envs/canary-vae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Class

In [2]:
from torch.utils.data import DataLoader
from data_class import SongDataSet_Image

train_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/test"
test_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/train"

train_dataset = SongDataSet_Image(train_dir, num_classes=21, psuedo_labels_generated=False)
test_dataset = SongDataSet_Image(test_dir, num_classes=21, psuedo_labels_generated=False)

collate_fn = CollateFunction(segment_length=1000)  # Adjust the segment length if needed

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

## Define Linear Classifier and Train

In [3]:
from linear_probe import LinearProbeModel, LinearProbeTrainer

classifier_model = LinearProbeModel(num_classes=21, model_type="neural_net", model=tweety_bert_model, freeze_layers=True, layer_num=-1, layer_id="attention_output", classifier_dims=384)
classifier_model = classifier_model.to(device)

In [4]:
trainer = LinearProbeTrainer(model=classifier_model, train_loader=train_loader, test_loader=test_loader, device=device, lr=1e-3, plotting=True, batches_per_eval=100, desired_total_batches=1e4, patience=4)
trainer.train()

Batch 100: FER = 50.39%, Train Loss = 1.9892, Val Loss = 2.1295
Batch 200: FER = 51.49%, Train Loss = 1.8678, Val Loss = 2.0234
Batch 300: FER = 49.62%, Train Loss = 2.1090, Val Loss = 2.0318


KeyboardInterrupt: 

## Analyze

In [ ]:
from linear_probe import ModelEvaluator

evaluator = ModelEvaluator(classifier_model, test_loader)
class_frame_error_rates, total_frame_error_rate = evaluator.validate_model_multiple_passes(num_passes=1, max_batches=1250)
evaluator.save_results(class_frame_error_rates, total_frame_error_rate, '/home/george-vengrovski/Documents/projects/tweety_bert_paper/results/test')

Evaluating: 100%|██████████| 133/133 [00:17<00:00,  7.63batch/s]
